In [16]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
import json
import requests
from ps_wd import pswd
from api_key import apikey
from pandas import json_normalize

In [17]:
dow_csv = "Resources/DowJonesIndex.csv"
dow_df = pd.read_csv(dow_csv)
dow_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1/8/2016,16519.16992,16651.89063,16314.57031,16346.45020,16346.45020,141850000
1,1/11/2016,16358.70996,16461.84961,16232.03027,16398.57031,16398.57031,127790000
2,1/12/2016,16419.10938,16591.34961,16322.07031,16516.22070,16516.22070,117480000
3,1/13/2016,16526.63086,16593.50977,16123.20020,16151.41016,16151.41016,153530000
4,1/14/2016,16159.00977,16482.05078,16075.12012,16379.04981,16379.04981,158830000


In [18]:
new_dow_df = dow_df[['Date', 'Open', 'Close']].copy()
new_dow_df.head()

,Date,Open,Close
0,1/8/2016,16519.16992,16346.45020
1,1/11/2016,16358.70996,16398.57031
2,1/12/2016,16419.10938,16516.22070
3,1/13/2016,16526.63086,16151.41016
4,1/14/2016,16159.00977,16379.04981


In [19]:
new_dow_df['Change'] = new_dow_df['Open'] - new_dow_df['Close']
new_dow_df.head()

,Date,Open,Close,Change
0,1/8/2016,16519.16992,16346.45020,172.71972
1,1/11/2016,16358.70996,16398.57031,-39.86035
2,1/12/2016,16419.10938,16516.22070,-97.11132
3,1/13/2016,16526.63086,16151.41016,375.22070
4,1/14/2016,16159.00977,16379.04981,-220.04004


In [23]:
!pip install psycopg2

In [24]:
engine = create_engine('postgresql://postgres:'+ pswd + '@localhost:5432/dow_jones_db')
connection = engine.connect()


In [25]:
engine.table_names()

['dow_jones', 'news_titles']

In [28]:
new_dow_df.to_sql(name='dow_jones', con=engine, if_exists='replace', index=False)

In [29]:
pd.read_sql_query('select * from dow_jones', con=engine).head()

,Date,Open,Close,Change
0,1/8/2016,16519.16992,16346.45020,172.71972
1,1/11/2016,16358.70996,16398.57031,-39.86035
2,1/12/2016,16419.10938,16516.22070,-97.11132
3,1/13/2016,16526.63086,16151.41016,375.22070
4,1/14/2016,16159.00977,16379.04981,-220.04004


In [ ]:
# URL of page to be scraped
url = 'https://www.cnn.com/article/sitemap-2016-1.html'

In [ ]:
# Retrieve page with the requests module
response = requests.get(url)

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# Examine the results, then determine element that contains sought info
print(soup.prettify())

In [ ]:
# results are returned as an iterable list
title_results = soup.find_all('span', class_="sitemap-link")
title_results

In [ ]:
# Loop through returned results
title_list = []
for result in title_results:
    title = result.text
    title_list.append(title)
    print(title)

In [ ]:
len(title_list)

In [ ]:
title_list[2237]

In [ ]:
title_df = DataFrame(title_list, columns =['Title'])
title_df

In [ ]:
# results are returned as an iterable list
date_results = soup.find_all('span', class_="date")
date_results

In [ ]:
# Loop through returned results
date_list = []
for result in date_results:
    date = result.text
    date_list.append(date)
    print(date)
   
    

In [ ]:
date_df = DataFrame(date_list, columns =['Date'])
date_df

In [ ]:
title_df['tmp'] = 1
date_df['tmp'] = 1

combined_news_df = pd.merge(title_df, date_df, on=['tmp'])
combined_news_df = combined_news_df.drop('tmp', axis=1)
combined_news_df

In [ ]:
concat_news_df = pd.concat([title_df, date_df ])
concat_news_df

In [ ]:
# URL of page to be scraped
# https://www.cnn.com/article/sitemap-2016-1.html
#url = 'https://www.cnn.com/article/sitemap-' + year + '-' + month + '.html'

In [ ]:
# Retrieve page with the requests module
#response = requests.get(url)

In [ ]:
# years = [2016, 2017, 2018, 2019, 2020]
# years = np.arange(2016, 2021)
# month = np.arange(1, 13)

In [16]:
years = np.arange(2016, 2021)
months = np.arange(1, 13)

title_list = []
date_list = []

for year in years:
    for month in months: 
  
        response = requests.get("https://www.cnn.com/article/sitemap-" + str(year) + '-' + str(month) + ".html")
  
        soup = BeautifulSoup(response.text, 'html.parser')
  
        date_results= soup.find_all('span', class_="date")
        title_results = soup.find_all('span', class_="sitemap-link")
        
        i = 0
        for result in title_results:
            title = result.text
            if title.find ('Trump') != -1:
                title_list.append(title)
                date_list.append(date_results[i].text)
            i+=1

In [19]:
title_list

['Jerry Falwell Jr. cites his father in Trump endorsement',
 'CNN/WMUR poll: Trump, Sanders still up in New Hampshire',
 'Major Iowa poll shows Trump, Clinton holding narrow leads',
 "Trump on blue collar appeal: My private jet is 'the ultimate bus'",
 'Why these Iowans are choosing Donald Trump over Ted Cruz',
 'In Iowa, Trump plays president in triumphant campaign stop',
 'Trump thanks evangelicals for their support',
 "Donald Trump: Ted Cruz is an 'anchor baby'",
 'Campaigns lower expectations in Iowa, brace for Trump win',
 'In Iowa, Cruz shifts focus from Trump to Rubio',
 "Cruz, Trump, Rubio? Who gets the 'Iowa bump'?",
 'Falwell endorses Trump? Satire is dead',
 "Political Prediction Market: Trump's chances for nomination at 50%",
 'Debate coach:  Sorry, Donald Trump, you lose',
 'Who won the Iowa GOP debate...besides Donald Trump?',
 "Donald Trump: Fox News 'apologized' over debate statement",
 "Donald Trump's sons go hunting in Iowa",
 'Falwell calls out Roger Ailes, Fox News 

In [20]:
len(title_list)

26725

In [21]:
title_list[26724]

"GOP silence on Trump's false election claims recalls McCarthy era"

In [22]:
date_list

['2016-01-31',
 '2016-01-31',
 '2016-01-31',
 '2016-01-31',
 '2016-01-31',
 '2016-01-30',
 '2016-01-30',
 '2016-01-30',
 '2016-01-30',
 '2016-01-30',
 '2016-01-30',
 '2016-01-29',
 '2016-01-29',
 '2016-01-29',
 '2016-01-29',
 '2016-01-29',
 '2016-01-29',
 '2016-01-29',
 '2016-01-29',
 '2016-01-29',
 '2016-01-29',
 '2016-01-29',
 '2016-01-28',
 '2016-01-28',
 '2016-01-28',
 '2016-01-28',
 '2016-01-28',
 '2016-01-28',
 '2016-01-28',
 '2016-01-28',
 '2016-01-28',
 '2016-01-28',
 '2016-01-28',
 '2016-01-28',
 '2016-01-28',
 '2016-01-28',
 '2016-01-28',
 '2016-01-28',
 '2016-01-28',
 '2016-01-27',
 '2016-01-27',
 '2016-01-27',
 '2016-01-27',
 '2016-01-27',
 '2016-01-27',
 '2016-01-27',
 '2016-01-27',
 '2016-01-27',
 '2016-01-27',
 '2016-01-27',
 '2016-01-27',
 '2016-01-27',
 '2016-01-27',
 '2016-01-27',
 '2016-01-27',
 '2016-01-27',
 '2016-01-27',
 '2016-01-26',
 '2016-01-26',
 '2016-01-26',
 '2016-01-26',
 '2016-01-26',
 '2016-01-26',
 '2016-01-26',
 '2016-01-26',
 '2016-01-26',
 '2016-01-

In [24]:
cnn_df = pd.DataFrame({"Date": date_list, "Title":title_list})
cnn_df.head()

,Date,Title
0,2016-01-31,Jerry Falwell Jr. cites his father in Trump en...
1,2016-01-31,"CNN/WMUR poll: Trump, Sanders still up in New ..."
2,2016-01-31,"Major Iowa poll shows Trump, Clinton holding n..."
3,2016-01-31,Trump on blue collar appeal: My private jet is...
4,2016-01-31,Why these Iowans are choosing Donald Trump ove...


In [30]:
cnn_df.to_sql(name='news_titles', con=engine, if_exists='replace', index=False)

In [31]:
pd.read_sql_query('select * from news_titles', con=engine).head()

,Date,Title
0,2016-01-31,Jerry Falwell Jr. cites his father in Trump en...
1,2016-01-31,"CNN/WMUR poll: Trump, Sanders still up in New ..."
2,2016-01-31,"Major Iowa poll shows Trump, Clinton holding n..."
3,2016-01-31,Trump on blue collar appeal: My private jet is...
4,2016-01-31,Why these Iowans are choosing Donald Trump ove...


In [8]:
# Save config information
crime_url = "https://api.usa.gov/crime/fbi/sapi/api/data/supplemental/states/CA/property_type/2016/2019?API_KEY=" + apikey

crime_response = requests.get(crime_url)
crime_json = crime_response.json()
print(json.dumps(crime_json, indent=4, sort_keys=True))


{
    "pagination": {
        "count": 4,
        "page": 0,
        "pages": 1,
        "per_page": 0
    },
    "results": [
        {
            "clothing_and_furs": 8853,
            "clothing_and_furs_recovered_count": 5626044,
            "clothing_and_furs_stolen_value": 113876603,
            "consumable_goods": 8853,
            "consumable_goods_recovered_count": 3004154,
            "consumable_goods_stolen_value": 31259439,
            "currency_notes_etc": 8853,
            "currency_notes_etc_recovered_count": 5402171,
            "currency_notes_etc_stolen_value": 249246157,
            "data_year": 2018,
            "firearms": 8853,
            "firearms_recovered_count": 972729,
            "firearms_stolen_value": 18832089,
            "household_goods": 8853,
            "household_goods_recovered_count": 1354054,
            "household_goods_stolen_value": 41489928,
            "jewelry_and_precious_metals": 8853,
            "jewelry_and_precious_metals_recovered

In [26]:
# import json_normalize
# in this dataset, the data to extract is under 'results'
crime_df = json_normalize(crime_json, 'results')
crime_df


,office_equipment,miscellaneous,not_specified,consumable_goods,locally_stolen_motor_vehicles,firearms,livestock,household_goods,jewelry_and_precious_metals,currency_notes_etc,...,consumable_goods_recovered_count,locally_stolen_motor_vehicles_recovered_count,firearms_recovered_count,livestock_recovered_count,household_goods_recovered_count,jewelry_and_precious_metals_recovered_count,currency_notes_etc_recovered_count,clothing_and_furs_recovered_count,telvisions_radio_stereos_etc_recovered_count,data_year
0,8853,8853,8853,8853,8853,8853,8853,8853,8853,8853,...,3004154,749684169,972729,111508,1354054,10771634,5402171,5626044,3827349,2018
1,8904,8904,0,8904,8904,8904,8904,8904,8904,8904,...,3289894,706177670,916606,102456,1570109,5704667,3641961,6272342,2605290,2016
2,8853,8853,8853,8853,8853,8853,8853,8853,8853,8853,...,3052947,687681240,1509155,144162,1149992,5305881,6228195,6106406,2670774,2019
3,8807,8807,8807,8807,8807,8807,8807,8807,8807,8807,...,2337220,744673819,1418204,88755,1550143,5634246,3146725,6210484,2487282,2017


In [27]:
crime_df.to_sql(name='crime_data', con=engine, if_exists='replace', index=False)

In [28]:
pd.read_sql_query('select * from crime_data', con=engine).head()

,office_equipment,miscellaneous,not_specified,consumable_goods,locally_stolen_motor_vehicles,firearms,livestock,household_goods,jewelry_and_precious_metals,currency_notes_etc,...,consumable_goods_recovered_count,locally_stolen_motor_vehicles_recovered_count,firearms_recovered_count,livestock_recovered_count,household_goods_recovered_count,jewelry_and_precious_metals_recovered_count,currency_notes_etc_recovered_count,clothing_and_furs_recovered_count,telvisions_radio_stereos_etc_recovered_count,data_year
0,8853,8853,8853,8853,8853,8853,8853,8853,8853,8853,...,3004154,749684169,972729,111508,1354054,10771634,5402171,5626044,3827349,2018
1,8904,8904,0,8904,8904,8904,8904,8904,8904,8904,...,3289894,706177670,916606,102456,1570109,5704667,3641961,6272342,2605290,2016
2,8853,8853,8853,8853,8853,8853,8853,8853,8853,8853,...,3052947,687681240,1509155,144162,1149992,5305881,6228195,6106406,2670774,2019
3,8807,8807,8807,8807,8807,8807,8807,8807,8807,8807,...,2337220,744673819,1418204,88755,1550143,5634246,3146725,6210484,2487282,2017
